# Senstivity study for the `eDrone` model - `RMSS2_V3`

### This model is based on work by: Saltelli
Great book (game changer for myself) --> https://www.amazon.com/Sensitivity-Analysis-Practice-Assessing-Scientific/dp/0470870931/ref=sr_1_1?dchild=1&qid=1626197470&refinements=p_27%3AAndrea+Saltelli&s=books&sr=1-1&text=Andrea+Saltelli <br>
Sensitivity Analysis in Practice: A Guide to Assessing Scientific Models 1st Edition <br>
by Andrea Saltelli (Author), Stefano Tarantola (Author), Francesca Campolongo (Author), Marco Ratto (Author)

In [2]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')

import pyNetLogo

#Import the sampling and analysis modules for a Sobol variance-based
#sensitivity analysis
from SALib.sample import saltelli
from SALib.analyze import sobol

### Variables - format:[variable-name, [low, step, high]
The following varaibles are avalible for use. Exersized variables bold <br>
["charger-network" true false]<br>
["has-charge-station" [0 0.01 1]]<br>
**["service-range" [1 1 170]]<br>**
**["chance-of-medical-event" [0 1.0E-4 0.01]]**<br>
**["chance-of-standard" [0 0.01 1]]**<br>
_["chance-of-emergency" [0 0.01 1]] - caluclated in netlogo (1-chance-of-emergency)_<br>
["activate-hospitals" true false]<br>
["drone-types" "444444444444444444444"] <br>
["replacebattery?" true false]<br>
["maxpayloadweight" [4 1 150]]<br>
**["meanpayloadweight" [1 1 150]]**<br>
**["startup-costs" [50000 10000 1000000]]**<br>
**["maintenace-cost-per-hour" [0 10 1000]]**<br>
["discharge_adjustment" [0 1 400]]<br>
**["chance_of_typea" [0 0.01 1]]**<br>
**["chance_of_typeb" [0 0.01 1]]**<br>
**["chance_of_typec" [0 0.01 1]]**<br>
**["number_of_drones" [1 1 30]]**<br>
**["number_of_eDrones" [0 1 3]]**<br>

### Responses 
- mean avgResponseTimeStandard
- mean avgResponseTimeEmergency
- length tmpList (queue Length) - no used now
- fitnessFunc
- totalOperatingCosts

In [3]:
# no need of drone type variables chance-of-x
problem = {
  'num_vars': 11,
  'names': ['service-range',
            'chance-of-medical-event',
            'chance-of-standard',
            'startup-costs',
            'meanpayloadweight',
            'maintenace-cost-per-hour',
            'chance_of_typea',
            'chance_of_typeb',
            'chance_of_typec',
            'number_of_drones',
            'number-of-eDrones'],
  'bounds': [[20, 120],
             [1e-4, .01],
             [.01, 1],
             [1e4, 2e6],
             [2, 20],
             [10, 50],
             [0, 1],
             [0, 1],
             [0, 1],
             [4, 10],
             [0, 3.5]]
}

with open('problem', 'w') as convert_file: 
     convert_file.write(json.dumps(problem))

In [4]:
n = 2048 #2^numbVars
param_values = saltelli.sample(problem, n, calc_second_order=True)
np.save('param_values', param_values)

In [5]:
# start the cluster -> ipcluster start -n 2
import ipyparallel
client = ipyparallel.Client()
client.ids

[0, 1, 2]

In [6]:
direct_view = client[:]

In [7]:
import os

#Push the current working directory of the notebook to a "cwd" variable on the engines that can be accessed later
direct_view.push(dict(cwd=os.getcwd()))

<AsyncResult: _push>

In [17]:
%%px
import os
os.chdir(cwd)

import pyNetLogo
import pandas as pd
#netlogo = pyNetLogo.NetLogoLink(gui=False)
path = "/srv/share/NetLogo_6.2.0"
ver = "6.2"
netlogo = pyNetLogo.NetLogoLink(netlogo_home = path, netlogo_version=ver, gui=False)

### Check here !!!!!! ########
netlogo.load_model('RMSS2_V3_eDrones.nlogo') #works
#netlogo.load_model('RMSS2_V2_2D_w_batt') #works
#netlogo.load_model('RMSS2_V2_2D_chg_net')
#netlogo.load_model('RMSS2_V2_2D_no_batt_net')
numbTicks = 43200 # approx. one month = 43200 minutes

### Note: Had to move the 'problem' dict inside the func... _probably an issue with it not being published to the cluster._

In [18]:
def simulation(experiment):
    
    problem = {
      'num_vars': 11,
      'names': ['service-range',
                'chance-of-medical-event',
                'chance-of-standard',
                'startup-costs',
                'meanpayloadweight',
                'maintenace-cost-per-hour',
                'chance_of_typea',
                'chance_of_typeb',
                'chance_of_typec',
                'number_of_drones',
                'number-of-eDrones'],
      'bounds': [[20, 120],
                 [1e-4, .01],
                 [.01, 1],
                 [1e4, 2e6],
                 [2, 20],
                 [10, 50],
                 [0, 1],
                 [0, 1],
                 [0, 1],
                 [4, 10],
                 [0, 3.5]]
    }

    #Set the input parameters
    for i, name in enumerate(problem['names']):
        if name == 'random-seed':
            #The NetLogo random seed requires a different syntax
            netlogo.command('random-seed {}'.format(experiment[i]))
        else:
            #Otherwise, assume the input parameters are global variables
            netlogo.command('set {0} {1}'.format(name, experiment[i]))

    netlogo.command('setup')
    
    #Run for XXXX ticks and return the response values
    response = netlogo.repeat_report(['arts_out','arte_out', 'fitnessFunc', 'totalOperatingCosts', 'drone-types'], numbTicks)

    # build results table
    results = pd.Series([response['arts_out'].values.mean(),
                         response['arte_out'].values.mean(),
                         response['fitnessFunc'].values.mean(),
                         response['totalOperatingCosts'].values.mean(),
                         response['drone-types'].values.max()],
                         index=['arts_out', 'arte_out','fitnessFunc','totalOperatingCosts','drone-types'])

    return results

In [ ]:
%%time 
lview = client.load_balanced_view()
results = pd.DataFrame(lview.map_sync(simulation, param_values))

In [11]:
## set fitness to 1e6 if response times are 0 - kill
df = results
df.loc[df.arts_out == 0, ['fitnessFunc']] = '1e4'
df.loc[df.arte_out == 0, ['fitnessFunc']] = '1e4'

In [12]:
## adjust for in or out testing
from datetime import *
filename = 'D_drones_sobal-' + datetime.now().strftime("%Y-%m-%d-%H-%M-%S")+".csv"
df.to_csv('results/' + filename)

In [ ]:
# release netlogo
#%%px
#netlogo.kill_workspace()